In [3]:
import pandas as pd

In [4]:
dataset_path = "C:/Users/stuti/Documents/CS410/Comparative-sentiment-analysis/IMDBDataset.csv"
dataset = pd.read_csv(dataset_path)
# rename the Column names to Text and Sentiment
dataset.columns = ['Text', 'Sentiment']
# remove the rows with missing values
dataset = dataset.dropna()


dataset.head()

,Text,Sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Process the text

In [5]:
import pandas as pd
import requests
from io import StringIO
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string
from collections import Counter
import tqdm
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer as ps
def process_text(text):
     # transform to lower case
    text = text.lower()
    # get the sentence with the most number of words
    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove excess white spaces from the text and get words list
    words_list = word_tokenize(text)
    # Remove stopwords
    stop_words = stopwords.words('english')
    words_list = [word for word in words_list if word not in stop_words]
    # join words seperated by whitespace
    modified_text = ' '.join(words_list)
    return modified_text, words_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stuti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stuti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# retain only 1000 random rows from the dataset
dataset = dataset.sample(1000)
# reset the index
dataset = dataset.reset_index(drop=True)
dataset.head()

,Text,Sentiment
0,"Just recently, I've been obsessing over and an...",negative
1,I cannot believe how this atrocity managed to ...,negative
2,"The Stooges are back and funnier than ever. ""B...",positive
3,"Damn, was that a lot to take in. I was pretty ...",positive
4,"It seems no matter what I see her in, Christin...",negative


In [7]:
# for each review, process the text
for i in range(len(dataset)):
    dataset['Text'][i], words_list = process_text(dataset['Text'][i])

C:\Users\stuti\AppData\Local\Temp\ipykernel_21788\3608623935.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset['Text'][i], words_list = process_text(dataset['Text'][i])


In [8]:
print(len(dataset))
true_sentiments = dataset['Sentiment'].values 

1000


## OpenAI sentiment analysis and evaluation

In [1]:
import openai


In [57]:
import time
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
def analyze_sentiment_openai(text):
    start_time = time.time()

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a sentiment analysis assistant. Classify the sentiment of the following text as positive or negative. Return only the sentiment label in english."},
        {"role": "user", "content": text}
        ]
    )
    end_time = time.time()
    response_time = end_time - start_time
    sentiment = completion.choices[0].message
    if "content" in sentiment:
        sentiment = sentiment['content']
    sentiment = str(sentiment).lower()
    if "positive" in sentiment:
        sentiment = "positive"
    elif "negative" in sentiment:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    return sentiment, response_time



In [37]:
sentiment, resps_time = analyze_sentiment_openai("I love this movie")


In [38]:
print(sentiment, resps_time)

positive 0.5577874183654785


In [39]:
results = []
predicted_labels_openai = []

for true_sentiment, text in zip(true_sentiments, dataset['Text'].tolist()):
    # openai_sentiment, openai_time = analyze_sentiment_openai(text)
    openai_sentiment, openai_time = analyze_sentiment_openai(text)
    
    results.append({
        "Text": text,
        "True Sentiment": true_sentiment,
        "OpenAI Sentiment": openai_sentiment,
        "OpenAI Response Time (s)": openai_time
    })
    
    # predicted_labels_openai.append(openai_sentiment)
    predicted_labels_openai.append(openai_sentiment)

# Convert results to DataFrame for easier analysis and display
results_df = pd.DataFrame(results)
results_df.to_csv("results_openai.csv", index=False)

In [20]:
import json
# save the results to a CSV file
# Calculate precision, recall, and F1-score for each model
# metrics_openai = precision_recall_fscore_support(true_sentiments, predicted_labels_openai, average='weighted')
def calculate_metrics(true_sentiments, predicted_labels, output_file_path, results_df):

    metrics_openai = precision_recall_fscore_support(true_sentiments, predicted_labels, average='weighted')
    F1_score_openai = 2/(1/metrics_openai[0] + 1/metrics_openai[1])

    # Print metrics
    print("openai Precision:", metrics_openai[0])
    print("openai Recall:", metrics_openai[1])
    print("openai F1-Score:", F1_score_openai)

    # average response time
    average_response_time_openai = results_df['OpenAI Response Time (s)'].mean()
    print("Average Response Time:", average_response_time_openai)
    metrics = {
        "OpenAI Precision": metrics_openai[0],
        "OpenAI Recall": metrics_openai[1],
        "OpenAI F1-Score": F1_score_openai,
        "Average Response Time": average_response_time_openai,
    }
    with open(output_file_path, "w") as f:
        json.dump(metrics, f)


In [44]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
def plot(results_df):
    confidence_scores = results_df['OpenAI Confidence'].values
    plt.figure(figsize=(10, 6))
    sns.histplot(confidence_scores, bins=30, kde=True)
    plt.title('Distribution of Confidence Scores for Adversarial Examples')
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.show()


In [45]:
calculate_metrics(true_sentiments, predicted_labels_openai, "metrics_openai.json")

openai Precision: 0.908916470645099
openai Recall: 0.906
openai F1-Score: 0.9074558920188325
Average Response Time: 0.32751070117950437


c:\Users\stuti\Documents\CS410\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Adversial testing


In [47]:
import random
from nltk.corpus import wordnet

# Ensure nltk resources are available
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# Function to create a simple adversarial example by replacing words with their synonyms
def synonym_replacement(text, n=1):
    words = text.split()
    n = len(words) // 3
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = set()
        for syn in wordnet.synsets(random_word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        if len(synonyms) > 1:
            synonyms.discard(random_word)
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
            if num_replaced >= n: # Only replace up to n words
                break
    
    sentence = ' '.join(new_words)
    return sentence

# Select a few texts randomly and create their adversarial examples
sample_texts = dataset['Text']
adversarial_examples = [synonym_replacement(text) for text in sample_texts]

def remove_random_spaces(text, n=1):
    words = text.split()
    n = len(words) // 10
    new_words = words.copy()
    
    for i in range(n):
        random_index = random.randint(0, len(new_words)-1)
        new_words.pop(random_index)
    
    sentence = ' '.join(new_words)
    return sentence

adversarial_examples = [remove_random_spaces(text) for text in sample_texts]


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stuti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\stuti\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [48]:
print(adversarial_examples)

['spanish movie worried many lost many world idea spanish civil war history teacher big advantage could easily see film could confusing together pieces doubt would big spain made audiences may bit confused later filmbr br movie begins yearold carol mother returning mothers home town years gather coming spain new york city despite strangeness carol speaks reasonably good spanish much film adapts mother alone ill spill dont pick fact carols father fighting war side eventually lost bloody conflictbr br movie gets exceptional lovely music rather slow rambling pace isnt bad provided someone accept films way seen spanish films war compares well themnot better worsejust decent slice film left behind well nice coming age flickbr br psif dont know side republicans mostly socialists backed soviet union mexico well volunteers many countries including usa nationalists led franco aided portugal fascist states germany italy', 'generally find loretta young hard concerned ladylike wrong ways lyrical f

In [50]:
results_adversial = []
predicted_labels_openai_adversial = []

for true_sentiment, text in zip(true_sentiments, adversarial_examples):
    # openai_sentiment, openai_time = analyze_sentiment_openai(text)
    openai_sentiment,openai_time = analyze_sentiment_openai(text)
    
    results_adversial.append({
        "Text": text,
        "True Sentiment": true_sentiment,
        "OpenAI Sentiment": openai_sentiment,
        "OpenAI Response Time (s)": openai_time,
    })
    
    # predicted_labels_openai.append(openai_sentiment)
    predicted_labels_openai_adversial.append(openai_sentiment)

# Convert results to DataFrame for easier analysis and display
results_df_adversial = pd.DataFrame(results_adversial)
results_df_adversial.to_csv("results_openai_adversial.csv", index=False)

In [51]:
calculate_metrics(true_sentiments, predicted_labels_openai_adversial, "metrics_openai_adversial30.json")

openai Precision: 0.8896974419916209
openai Recall: 0.887
openai F1-Score: 0.8883466733220968
Average Response Time: 0.32751070117950437


c:\Users\stuti\Documents\CS410\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
from deep_translator import GoogleTranslator

def translate(text):
    translator = GoogleTranslator(source='auto', target='es')
    translated_text = translator.translate(text)
    return translated_text


In [12]:
dataset_translated = dataset.copy()

for i in range(len(dataset_translated)):
    dataset_translated['Text'][i] = translate(dataset_translated['Text'][i])



C:\Users\stuti\AppData\Local\Temp\ipykernel_21788\462684534.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset_translated['Text'][i] = translate(dataset_translated['Text'][i])


In [32]:
print(dataset_translated.head())

                                                Text Sentiment
0  Recientemente me he obsesionado anticipando la...  negative
1  creer atrocidad manejado capturar corazones me...  negative
2  títeres atrás más divertido nunca sin novia no...  positive
3  Maldita sea, muchos se quedaron bastante hipno...  positive
4  parece importar ver christina ricci parece com...  negative


In [58]:
results_lang = []
predicted_labels_openai_lang = []

for true_sentiment, text in zip(true_sentiments, dataset_translated['Text'].tolist()):
    # openai_sentiment, openai_time = analyze_sentiment_openai(text)
    try:
        openai_sentiment, openai_time = analyze_sentiment_openai(text)

    except Exception as e:
        print(e)
        openai_sentiment, openai_time = 'neutral', 0, 0
    results_lang.append({
        "Text": text,
        "True Sentiment": true_sentiment,
        "OpenAI Sentiment": openai_sentiment,
        "OpenAI Response Time (s)": openai_time,
    })
    
    # predicted_labels_openai.append(openai_sentiment)
    predicted_labels_openai_lang.append(openai_sentiment)

# Convert results to DataFrame for easier analysis and display
results_df_lang = pd.DataFrame(results_lang)
results_df_lang.to_csv("results_openai_lang.csv", index=False)

In [59]:
print(predicted_labels_openai_lang)
calculate_metrics(true_sentiments, predicted_labels_openai_lang, "metrics_openai_lang.json", results_df_lang)

['negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'po

c:\Users\stuti\Documents\CS410\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
